# Getting Started with iso8583sim

This notebook introduces the iso8583sim library for working with ISO 8583 financial messages.

## What is ISO 8583?

ISO 8583 is the international standard for financial transaction card-originated messages. It's used by:
- Payment networks (Visa, Mastercard, etc.)
- ATM systems
- Point-of-sale terminals
- Banking systems

## Installation

```bash
pip install iso8583sim
```

## Quick Start

Let's start by importing the main components:

In [ ]:
import sys

sys.path.insert(0, "..")  # For development - remove in production

from iso8583sim.core.builder import ISO8583Builder
from iso8583sim.core.parser import ISO8583Parser
from iso8583sim.core.types import ISO8583Message
from iso8583sim.core.validator import ISO8583Validator

## Building Your First Message

Let's create a simple authorization request (MTI 0100):

In [ ]:
# Create a builder instance
builder = ISO8583Builder()

# Define message fields
message = ISO8583Message(
    mti="0100",  # Authorization Request
    fields={
        0: "0100",  # MTI
        2: "4111111111111111",  # Primary Account Number (PAN)
        3: "000000",  # Processing Code (purchase)
        4: "000000001000",  # Transaction Amount ($10.00)
        11: "123456",  # System Trace Audit Number
        41: "TERM0001",  # Card Acceptor Terminal ID
        42: "MERCHANT123456 ",  # Card Acceptor ID
    },
)

# Build the raw message
raw_message = builder.build(message)
print(f"Raw message ({len(raw_message)} bytes):")
print(raw_message)

## Understanding the Message Structure

An ISO 8583 message consists of:

1. **MTI (Message Type Indicator)** - 4 digits identifying the message type
2. **Bitmap** - Indicates which fields are present
3. **Data Elements** - The actual field values

Let's break down our message:

In [ ]:
# Parse the MTI
mti = raw_message[:4]
print(f"MTI: {mti}")
print(f"  - Version: {mti[0]} (0=1987, 1=1993, 2=2003)")
print(f"  - Class: {mti[1]} (1=Authorization)")
print(f"  - Function: {mti[2]} (0=Request)")
print(f"  - Origin: {mti[3]} (0=Acquirer)")

# Parse the bitmap (16 hex chars = 64 bits for primary bitmap)
bitmap = raw_message[4:20]
print(f"\nBitmap: {bitmap}")

## Parsing Messages

Now let's parse the message back:

In [ ]:
# Create a parser instance
parser = ISO8583Parser()

# Parse the raw message
parsed = parser.parse(raw_message)

print(f"MTI: {parsed.mti}")
print("\nFields:")
for field_num, value in sorted(parsed.fields.items()):
    if field_num > 0:  # Skip MTI (field 0)
        print(f"  Field {field_num:3d}: {value}")

## Validating Messages

The validator checks message integrity and field formats:

In [ ]:
# Create a validator instance
validator = ISO8583Validator()

# Validate the parsed message
result = validator.validate_message(parsed)

print(f"Valid: {result.is_valid}")
if result.errors:
    print("Errors:")
    for error in result.errors:
        print(f"  - {error}")
if result.warnings:
    print("Warnings:")
    for warning in result.warnings:
        print(f"  - {warning}")

## Creating a Response

The builder can create responses from requests automatically:

In [ ]:
# Create a response to our authorization request
response = builder.create_response(
    parsed,
    response_fields={
        38: "ABC123",  # Authorization Code
        39: "00",  # Response Code (00 = Approved)
    },
)

print(f"Response MTI: {response.mti}")  # Should be 0110
print("\nResponse Fields:")
for field_num, value in sorted(response.fields.items()):
    if field_num > 0:
        print(f"  Field {field_num:3d}: {value}")

## Common MTI Values

| MTI | Description |
|-----|-------------|
| 0100 | Authorization Request |
| 0110 | Authorization Response |
| 0200 | Financial Request |
| 0210 | Financial Response |
| 0400 | Reversal Request |
| 0410 | Reversal Response |
| 0800 | Network Management Request |
| 0810 | Network Management Response |

## Common Response Codes (Field 39)

| Code | Description |
|------|-------------|
| 00 | Approved |
| 01 | Refer to issuer |
| 05 | Do not honor |
| 12 | Invalid transaction |
| 14 | Invalid card number |
| 51 | Insufficient funds |
| 54 | Expired card |
| 91 | Issuer unavailable |

## Next Steps

- **[02_parsing_messages.ipynb](02_parsing_messages.ipynb)** - Deep dive into message parsing
- **[03_building_messages.ipynb](03_building_messages.ipynb)** - Advanced message building
- **[04_network_specifics.ipynb](04_network_specifics.ipynb)** - VISA, Mastercard, and other networks
- **[05_emv_data.ipynb](05_emv_data.ipynb)** - Working with EMV/chip card data